In [ ]:
#SCUSATE PER I NOMI UN PO' A CAZZO DI CANE MA HO FATTO PER SBRIGARMI

import requests
import matplotlib.pyplot as plt
import numpy as np
import lxml.html
from lxml import etree
import xml.etree.ElementTree as ET
import string
import csv
import numpy as np
import time

def AbsolutePaths(tags):
    tmp=''
    for tag in tags:
        tmp+=tag.tag+'/'
    return str(tmp)    

def xpath_generator(C):
  X = []
  for link in C:
    #ritorna la get
    response = requests.get(link)
    #ritorna l'elemento html della pagina
    tree = lxml.html.fromstring(response.text)
    #ritorna la lista dei nodi foglia
    childs = tree.xpath('//*[not(child::*)]')
    #funzione che calcola il path assoluto radice-foglia come stringa
    #calcolo della lista che contiene gli xpath assoluti alle foglie della pagina 
    for child in childs:
      #ciascun elemento in X è una quadrupla (implementata come una lista di 4 elementi) formata da:
      #url della pagina, xpath di ciascun nodo della pagina, contenuto del nodo, peso del nodo
      xpath_element = []
      tags = child.xpath('ancestor::node()')
      path = AbsolutePaths(tags)
      path += str(child.tag)
      contenuto = child.text_content()
      xpath_element.append(link)
      xpath_element.append(path)
      xpath_element.append(contenuto)
      xpath_element.append(0)
      X.append(xpath_element)
  return X

#Ricerca la frequenza e la frequenza del contenuto di un certo xpath
def find_f(url, xpath, contenuto, lista):
  frequenza = 1
  f_contenuto = 1
  #lista dei url già visti nella ricerca
  url_visti = []
  url_visti.append(url)
  for new_path in lista:
    if new_path[0] not in url_visti:
      if xpath == new_path[1]:
        url_visti.append(new_path[0])
        frequenza += 1
        if contenuto == new_path[2]:
          f_contenuto += 1  
  return frequenza, f_contenuto

#lista = [(url,xpath,contenuto,peso), (....), (....), .... ]
def calcolo_peso_xpath(lista, num_pagine):
  for xpath in lista:
    #prendo xpath
    x = xpath[1]
    url = xpath[0]
    contenuto = xpath[2]
    #calcolo la frequenza e la frequenza del contenuto
    frequenza, f_contenuto = find_f(url, x, contenuto, lista)
    #calcolo della informatività
    info = 1 - (f_contenuto/num_pagine)
    #calcolo del peso del xpath
    peso = frequenza * info
    #inserisco il peso nella lista
    xpath[3] = peso
    
def somma_pesi_per_pagina(X):
  url_pesi = {}
  for e in X:
    k = e[0]
    if k not in url_pesi.keys():
      url_pesi[k] = e[3]
    else:
      url_pesi[k] += e[3]
  return d

#Trova la pagina (ritornando il suo url) con la somma dei pesi maggiore
def find_p(X, M):
  calcolo_peso_xpath(X, M)
  pagine_con_pesi = somma_pesi_per_pagina(X)
  peso_max = 0
  pagina_max = ''
  for k in pagine_con_pesi.keys():
    peso_pagina = pagine_con_pesi[k]
    if peso_pagina > peso_max:
      pagina_max = k
      peso_max = peso_pagina
  return pagina_max

#Crea un dizionario con chiave xpath e valore il contenuto
def build_dict(X,p):
  d = {}
  for e in X:
    if e[0] == p:
      if e[1] not in d:
        xpath = e[1]
        content = e[2]
        d[xpath] = content
  return d

#Rimuove la pagina da X e gli Xpath associati ad essa
def remove_p(X, p):
  X1 = []
  for e in X:
    if e[0] != p:
      X1.append(e)
  #elimino gli xpath contenuto in p nelle altre pagine
  X2 = []
  dict_xpath_content = build_dict(X,p)
  for e in X1:
    if e[1] not in dict_xpath_content:
      X2.append(e)
    elif e[2] != dict_xpath_content[e[1]]:
        X2.append(e)  
  return X2

#Main
def page_annotation(C, K):
  X = xpath_generator(C)
  S = []
  while X!=[] and len(S)<=K:
    p = find_p(X, len(C))
    S.append(p)
    X = remove_p(X, p)
  return S